In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import Normalizer, MinMaxScaler,StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, mean_squared_error, accuracy_score, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
data = pd.read_csv('../input/insurance-premium-prediction/insurance.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.sex.value_counts()

In [ ]:
data['sex'] = data['sex'].replace({'female':'F','male':'M'})

In [ ]:
data.head()

In [ ]:
sns.catplot(x='sex',hue='smoker',col='region',data=data,kind='count')

In [ ]:
data.head()

In [ ]:
sns.countplot(x='sex',data=data)

In [ ]:
sns.catplot(x='sex',hue='smoker',data=data,kind='count')

In [ ]:
sns.distplot(data['age'])

In [ ]:
sns.countplot(x='children',data=data)

In [ ]:
sns.catplot(x='region',hue='smoker',data=data,kind='count')

In [ ]:
data.head()

In [ ]:
sns.pairplot(data=data,hue='sex')

In [ ]:
sns.catplot(x='smoker',y='age',hue='region',col='sex',data=data,kind='box')

In [ ]:
sns.catplot(x='sex',y='expenses',data=data,kind='box')

In [ ]:
sns.catplot(x='smoker',y='expenses',hue='region',data=data,kind='box')

In [ ]:
df = pd.get_dummies(data)

In [ ]:
data.head()

In [ ]:
df.head()

In [ ]:
X = df.drop(['expenses'],axis=1)

In [ ]:
y = df['expenses']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
NM = Normalizer()

In [ ]:
dfa = pd.DataFrame(NM.fit_transform(X),columns=X.columns)

In [ ]:
dfa.head()

In [ ]:
dfa.shape

In [ ]:
XX_train,X_test,yy_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_cv,y_train,y_cv = train_test_split(XX_train,yy_train,test_size=0.2,random_state=42)

In [ ]:
RF = RandomForestRegressor()
RF_param = {'n_estimators':range(10,200,100),'max_depth':range(1,50)}
RF_model = RandomizedSearchCV(RF,RF_param,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
RF_model.fit(X_cv,y_cv)

In [ ]:
print(RF_model.best_score_)
print(RF_model.best_estimator_)
print(RF_model.best_params_)

In [ ]:
RF_Result = RandomForestRegressor(n_estimators=110,max_depth=3)
RF_Result.fit(X_train,y_train)
pred1 = RF_Result.predict(X_test)
np.sqrt(mean_squared_error(y_test,pred1))


In [ ]:
print(RF_Result.score(X_test,y_test))
print(mean_absolute_error(y_test,pred1))

In [ ]:
XB = XGBRegressor()

In [ ]:
XB.fit(X_train,y_train)
pred2=XB.predict(X_test)
np.sqrt(mean_squared_error(pred2,y_test))

In [ ]:
print(XB.score(X_test,y_test))
print(mean_absolute_error(y_test,pred2))

In [ ]:
DTR = DecisionTreeRegressor()
DTR_para = GridSearchCV(DTR,param_grid={'min_samples_split':range(2,30)},scoring='neg_mean_squared_error',cv=6)
DTR_para.fit(X_cv,y_cv)
print(DTR_para.best_score_)
print(DTR_para.best_estimator_)
print(DTR_para.best_params_)

In [ ]:
DTR_result = DecisionTreeRegressor(min_samples_split=29,random_state=42)
DTR_result.fit(X_train,y_train)
pred3 = DTR_result.predict(X_test)
np.sqrt(mean_squared_error(y_test,pred3))

In [ ]:
print(DTR_result.score(X_test,y_test))
print(mean_absolute_error(y_test,pred3))

In [ ]:
LR = LinearRegression()
LR.fit(X_train,y_train)

In [ ]:
pred4 = LR.predict(X_test)

In [ ]:
print(LR.score(X_test,y_test))
print(np.sqrt(mean_squared_error(y_test,pred4)))

In [ ]:
print(mean_absolute_error(y_test,pred4))
print(LR.intercept_)

In [ ]:
Result = pd.DataFrame({"Algo":['RandomForestRegressor','XGBoost','DecisionTreeRegressor','LinearRegression'],
                      'Mean_Absolute_Error':[mean_absolute_error(y_test,pred1),mean_absolute_error(y_test,pred2),mean_absolute_error(y_test,pred3),mean_absolute_error(y_test,pred4)]})

In [ ]:
Result